In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import pandas_datareader.data as pd_web
from datetime import datetime
import yfinance as yf
yf.pdr_override()  # update pandas datareader (yahoo finance api)

import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from torch.utils.data import TensorDataset # 텐서데이터셋
from torch.utils.data import DataLoader # 데이터로더

device = torch.device('mps:0' if torch.backends.mps.is_available() else 'cpu')
print(f"{device} 사용 가능합니다")

mps:0 사용 가능합니다


In [2]:
df = pd.read_csv('./NVDA_110721_Final_1_fdr.csv', index_col='Date')
df['Close_y'] = df['Close']

seq_length = 40
batch = 16
train_size = int(len(df) * 0.7)
train_set = df[0:train_size]
test_set = df[train_size - seq_length:]

train_set.shape, test_set.shape

test_set
type(df.iloc[:, -1]), type(df.iloc[:, [-1]])

(pandas.core.series.Series, pandas.core.frame.DataFrame)

In [3]:
# Input scale
scaler_x = StandardScaler()
scaler_x.fit(train_set.iloc[:, :-1])  # 맨마지막에 test y 추가

train_set.iloc[:, :-1] = scaler_x.transform(train_set.iloc[:, :-1])
test_set.iloc[:, :-1] = scaler_x.transform(test_set.iloc[:, :-1])

# Output scale
scaler_y = StandardScaler()
scaler_y.fit(train_set.iloc[:, [-1]])

train_set.iloc[:, -1] = scaler_y.transform(train_set.iloc[:, [-1]])
test_set.iloc[:, -1] = scaler_y.transform(test_set.iloc[:, [-1]])

train_set

/Users/sihun/anaconda3/envs/madellion/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/sihun/anaconda3/envs/madellion/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/sihun/anaconda3/envs/madellion/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/sihun/anaconda3/envs/madellion/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated

,Close,MA5,MA20,MA60,RSI14,std,upperb,lowerb,%K,%D,SP500,NASDAQ_COMP,Close_y
Date,,,,,,,,,,,,,
2011/07/21,-0.613332,-0.619564,-0.601599,-0.560204,-1.183925,-0.472505,-0.596549,-0.606209,-0.582667,-1.136944,-1.380350,-1.290105,-0.613332
2011/07/22,-0.609155,-0.617047,-0.602458,-0.561271,-0.844317,-0.485486,-0.598451,-0.605842,-0.208151,-0.787392,-1.377541,-1.272344,-0.609155
2011/07/25,-0.612497,-0.614362,-0.603154,-0.562450,-1.054733,-0.489928,-0.599473,-0.606152,-0.270569,-0.379580,-1.395019,-1.284013,-0.612497
2011/07/26,-0.617509,-0.613859,-0.603871,-0.563840,-1.348803,-0.487934,-0.599966,-0.607131,-0.758910,-0.442847,-1.407662,-1.286080,-0.617509
2011/07/27,-0.625026,-0.614362,-0.605036,-0.565297,-1.732635,-0.479145,-0.600293,-0.609277,-1.532441,-0.918239,-1.469955,-1.340796,-0.625026
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018/07/13,-0.670242,-0.667866,-0.662168,-0.645826,-4.619692,1.982617,-0.674130,-0.646832,-1.910230,-2.069635,-4.470387,-3.351592,-0.670242
2018/07/16,-0.671110,-0.668030,-0.662856,-0.645543,-4.627021,1.665445,-0.675871,-0.646052,-1.916145,-2.058705,-4.470402,-3.351603,-0.671110
2018/07/17,-0.666854,-0.667962,-0.663316,-0.645174,-4.591807,1.337446,-0.677457,-0.644948,-1.887972,-2.047674,-4.470343,-3.351576,-0.666854


In [4]:
# 데이터셋 생성 함수
def build_dataset(time_series, seq_length):
    dataX = []
    dataY = []
    for i in range(0, len(time_series)-seq_length):
        _x = time_series[i:i+seq_length, :]
        _y = time_series[i+seq_length, [-1]]
        # print(_x, "-->",_y)
        dataX.append(_x)
        dataY.append(_y)

    return np.array(dataX), np.array(dataY)

trainX, trainY = build_dataset(np.array(train_set), seq_length)
testX, testY = build_dataset(np.array(test_set), seq_length)

trainX_tensor = torch.tensor(trainX)
trainY_tensor = torch.tensor(trainY)

testX_tensor = torch.tensor(testX)
testY_tensor = torch.tensor(testY)

dataset = TensorDataset(trainX_tensor, trainY_tensor)

dataloader = DataLoader(dataset,
                        batch_size=batch,
                        shuffle=True,
                        drop_last=True)  # 싸이즈 다르면 드롭?



In [5]:
# 설정값
data_dim = 5
hidden_dim = 10 
output_dim = 1 
learning_rate = 0.01
nb_epochs = 100

class Net(nn.Module):
    def __init__(self, input_dim, hidden_dim, seq_len, output_dim, layers):
        super(Net, self).__init__()
        self.hidden_dim = hidden_dim
        self.seq_len = seq_len
        self.output_dim = output_dim
        self.layers = layers

        self.lstm = nn.LSTM(input_dim, 
                            hidden_dim,
                            num_layers=layers,
                            batch_first=True)
        
        self.fc = nn.Linear(hidden_dim, output_dim, bias = True)

    def reset_hidden_state(self): 
        self.hidden = (
                torch.zeros(self.layers, self.seq_len, self.hidden_dim),
                torch.zeros(self.layers, self.seq_len, self.hidden_dim))
        
    # 예측을 위한 함수
    def forward(self, x):
        x, _status = self.lstm(x)
        x = self.fc(x[:, -1])
        return x

In [6]:
def train_model(model, train_df, num_epochs = None, lr = None, verbose = 10, patience = 10):
     
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr = learning_rate)
    nb_epochs = num_epochs
    
    # epoch마다 loss 저장
    train_hist = np.zeros(nb_epochs)

    for epoch in range(nb_epochs):
        avg_cost = 0
        total_batch = len(train_df)
        
        for batch_idx, samples in enumerate(train_df):

            x_train, y_train = samples
            
            # seq별 hidden state reset
            model.reset_hidden_state()
            
            # H(x) 계산
            outputs = model(x_train)
                
            # cost 계산
            loss = criterion(outputs, y_train)                    
            
            # cost로 H(x) 개선
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            avg_cost += loss/total_batch
               
        train_hist[epoch] = avg_cost        
        
        if epoch % verbose == 0:
            print('Epoch:', '%04d' % (epoch), 'train loss :', '{:.4f}'.format(avg_cost))
            
        # patience번째 마다 early stopping 여부 확인
        if (epoch % patience == 0) & (epoch != 0):
            
            # loss가 커졌다면 early stop
            if train_hist[epoch-patience] < train_hist[epoch]:
                print('\n Early Stopping')
                
                break
            
    return model.eval(), train_hist

In [7]:
net = Net(data_dim, hidden_dim, seq_length, output_dim, 1)
model, train_hist = train_model(net, dataloader, num_epochs = nb_epochs, lr = learning_rate, verbose = 20, patience = 10)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (640x13 and 5x40)